In [1428]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import xgboost as xgb
import math
import csv
import seaborn as sns
plt.style.use('fivethirtyeight')
%matplotlib inline

In [1429]:
train_a = pd.read_parquet('../../A/train_targets.parquet')
train_b = pd.read_parquet('../../B/train_targets.parquet')
train_c = pd.read_parquet('../../C/train_targets.parquet')

X_train_estimated_a = pd.read_parquet('../../A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../../B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../../C/X_train_estimated.parquet')

X_train_observed_a = pd.read_parquet('../../A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../../B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../../C/X_train_observed.parquet')

X_test_estimated_a = pd.read_parquet('../../A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../../B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../../C/X_test_estimated.parquet')

In [1430]:
X_train_estimated_c[X_train_estimated_c['date_forecast'] == '2023-01-26']

,date_calc,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,...,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms


In [1431]:
X_train_estimated_c.set_index('date_forecast', inplace=True)
X_train_estimated_c.drop('date_calc', axis=1, inplace=True)

In [1432]:
X_train_estimated_c.index.min()

Timestamp('2022-10-28 22:00:00')

In [1433]:
X_train_estimated_c.loc['2023-01-26']

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
date_forecast,,,,,,,,,,,,,,,,,,,,,


In [1434]:
def add_distance_to_solar_noon(df_in):
    """ 
        Adds the distance to solar noon as a feature. Solar noon is calculated by taking to time in which the sun elevation
        is at its highgest for the day. The distance to this point in time is calculated in minutes.

        Assumes datetime index. Must be before resampling.
    """

    df = df_in.copy()

    all_times = set(pd.date_range(start="00:00", end="23:45", freq='15T').time)
    valid_dates = df.groupby(df.index.date).filter(lambda group: set(group.index.time) == all_times)
    df = valid_dates

    daily_max_elevation = df.resample('D')['sun_elevation:d'].idxmax()
    daily_max_elevation = daily_max_elevation.to_frame()
    daily_max_elevation['sun_elevation:d'] = daily_max_elevation['sun_elevation:d'].dt.time
    daily_max_elevation.rename(columns={'sun_elevation:d' : 'distance_to_solar_noon'}, inplace=True)

    df = df.merge(daily_max_elevation, left_index=True, right_index=True, how='outer')
    df['distance_to_solar_noon'] = df.groupby(df.index.date)['distance_to_solar_noon'].ffill()

    df['solar_noon_timedelta'] = pd.to_timedelta(df['distance_to_solar_noon'].astype(str))
    df['solar_noon_datetime'] = df.index.normalize() + df['solar_noon_timedelta']
    df['distance_to_solar_noon'] = (df['solar_noon_datetime'] - df.index).dt.total_seconds().abs() / 60

    return df
    

In [1435]:
X_train_estimated_a.shape

(17576, 47)

In [1436]:
X_train_estimated_a.loc['2023-01-27']

KeyError: '2023-01-27'

In [ ]:
X_train_estimated_a = add_distance_to_solar_noon(X_train_estimated_a)
X_train_estimated_a.shape

ValueError: attempt to get argmax of an empty sequence